In [ ]:
# !pip install google-cloud-aiplatform[prediction]>=1.16.0 fastapi nvtabular git+https://github.com/NVIDIA-Merlin/models.git --user

# Buidling a custom Vertex AI endpoint for Merlin Query Tower

**IMPORTANT** Make sure you are running this notebook in a DLVM (e.g. tensorflow enterprise 2.8) to build the image

________
**This will not work in the training container**
________

Your output should look like this - you are going to use the query model endpoint to create a custom container

![](img/merlin-bucket.png)


In [1]:
# !gsutil cp -r $workflow_uri local_workflow

In [2]:
# !gsutil cp -r gs://spotify-beam-v3/merlin-processed/workflow/query-tower-model-22-12 local_model

#### END DEBUG

In [1]:
from datetime import datetime


PROJECT = 'wortz-project-352116'  # <--- TODO: CHANGE THIS
LOCATION = 'us-central1'
REGION = 'us-central1'
# path = 'gs://two-tower-models' #TODO change to your model directory
BUCKET = 'gs://spotify-jsw-mpd-2023'
REPOSITORY = 'merlin-spotify-cpr'
ARTIFACT_URI = f'{BUCKET}'
MODEL_DIR = f'{BUCKET}/query_model_merlin'
PREFIX = 'merlin-spotify'

### Run one time to locally copy the workflow for the image

In [2]:
# !mkdir workflow
# !gsutil cp -r $BUCKET/workflow workflow

In [3]:
# !gsutil mb -l us-central1 gs://wortz-project-bucket

In [4]:
# !echo y | docker container prune
# !echo y | docker image prune

### Set up repo and configure Docker (one-time)

In [5]:
# Create the repo if needed for the artifacts

! gcloud beta artifacts repositories create {REPOSITORY} \
    --repository-format=docker \
    --location=$REGION

ERROR: (gcloud.beta.artifacts.repositories.create) ALREADY_EXISTS: the repository already exists


In [6]:
! gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
gcloud credential helpers already registered correctly.


In [7]:
! rm -rf app
! mkdir app

### Dependency file
The first few are for the server handling traffic
Nvtabular was downgraded for this example, it may not be necessary in future versions

In [8]:
%%writefile app/requirements.txt
uvicorn[standard]==0.15.0
gunicorn==20.1.0
fastapi==0.68.1
google-cloud-aiplatform
merlin-models
nvtabular
gcsfs
google-cloud-storage

Writing app/requirements.txt


### Predictor module and class
This is an adaptation of the CPR examples

This was locally tested and created shortly after model creation in the [tensorflow-predict](tensorflow-predict.ipynb) notebook

In [9]:
%%writefile app/dataset_to_tensors.py

try:
    import cudf
except ImportError:
    cudf = None
import pandas as pd
import tensorflow as tf
from typing import Dict
from merlin.io import Dataset
import itertools


def cupy_array_to_tensor(array):
    return tf.experimental.dlpack.from_dlpack(array.reshape(-1, 1).toDlpack())

def numpy_array_to_tensor(array):
    return tf.convert_to_tensor(array.reshape(-1, 1))

def cudf_series_to_tensor(col) -> tf.Tensor:
    "Convert a cudf.Series to a TensorFlow Tensor with DLPack"
    if isinstance(col.dtype, cudf.ListDtype):
        values = col.list.leaves.values
        offsets = col.list._column.offsets.values
        row_lengths = offsets[1:] - offsets[:-1]
        return cupy_array_to_tensor(values), cupy_array_to_tensor(row_lengths)
    else:
        return cupy_array_to_tensor(col.values)

def pandas_series_to_tensor(col) -> tf.Tensor:
    if len(col) and pd.api.types.is_list_like(col.values[0]):
        values = pd.Series(itertools.chain(*col)).values
        row_lengths = col.map(len).values
        return numpy_array_to_tensor(values), numpy_array_to_tensor(row_lengths)
    else:
        return numpy_array_to_tensor(col.values)
        
    
def dataset_to_tensors(dataset: Dataset) -> Dict[str, tf.Tensor]:
    """Convert a DataFrame to Dict of Tensors"""
    df = dataset.to_ddf().compute()
    if isinstance(df, pd.DataFrame):
        col_to_tensor = pandas_series_to_tensor
    else:
        col_to_tensor = cudf_series_to_tensor
    return {
        column: col_to_tensor(df[column])
        for column in df.columns
    }

Writing app/dataset_to_tensors.py


In [309]:
%%writefile app/predictor.py

import nvtabular as nvt
import pandas as pd
import os
import json
import merlin.models.tf as mm
from nvtabular.loader.tf_utils import configure_tensorflow
configure_tensorflow()
import tensorflow as tf
import logging
from dataset_to_tensors import *
import time


# These are helper functions that ensure the dictionary input is in a certain order and types are preserved
# this is to get scalar values to appear first in the dict to not confuse pandas with lists https://github.com/pandas-dev/pandas/issues/46092
reordered_keys = ['pid',
                 'pl_name_src',
                 'pl_collaborative_src',
                 'pl_duration_ms_new',
                 'artist_name_pl',
                 'track_uri_pl',
                 'track_name_pl',
                 'album_name_pl']

# float_num_fix = None #['n_songs_pl','num_albums_pl','num_artists_pl','duration_ms_seed_pl']
# float_list_fix = ['pl_duration_ms_new']
    
# def fix_list_num_dtypes(num_list):
#     "this fixes lists of ints to list of floats converted in json input"
#     return [float(x) for x in num_list]

# def fix_num_dtypes(num):
#     "this fixes ints and casts to floats"
#     return float(num)

# def fix_types(k, v):
#     return v
    # if k in float_num_fix:
    #     return fix_num_dtypes(v)
    # if k in float_list_fix:
    #     return fix_list_num_dtypes(v)
    # else:
    #     return v

def create_pandas_instance(inputs):
    """
    Helper function to reorder the input to have a sclar first for pandas
    And fix the types converted when data is imported by fastAPI
    """
    if type(inputs[0]) == dict:
        header = inputs[0]
        reordered_header_dict = {k: header[k] for k in reordered_keys}
        pandas_instance = pd.DataFrame.from_dict(reordered_header_dict, orient='index').T

        if len(inputs) > 1:
            for ti in inputs[1:]:
                reordered_dict = {k: ti[k] for k in reordered_keys}
                pandas_instance = pandas_instance.append(pd.DataFrame.from_dict(reordered_dict, orient='index').T)
    else:
        for imps in inputs:
            input_dict = {'pid': imps[0],
                     'pl_name_src': imps[1],
                     'pl_collaborative_src': imps[2],
                     'pl_duration_ms_new': imps[3],
                     'artist_name_pl': imps[4],
                     'track_uri_pl': imps[5],
                     'track_name_pl': imps[6],
                     'album_name_pl': imps[7]}
            try: 
                pandas_instance.append(pd.DataFrame.from_dict(input_dict, orient='index').T)
            except NameError:
                pandas_instance = pd.DataFrame.from_dict(input_dict, orient='index').T
                
        
    return pandas_instance

class Predictor():
    """Interface of the Predictor class for Custom Prediction Routines.
    The Predictor is responsible for the ML logic for processing a prediction request.
    Specifically, the Predictor must define:
    (1) How to load all model artifacts used during prediction into memory.
    (2) The logic that should be executed at predict time.
    When using the default PredictionHandler, the Predictor will be invoked as follows:
      predictor.postprocess(predictor.predict(predictor.preprocess(prediction_input)))
    """
    def __init__(self):
        return
    
    def load(self, artifacts_uri):
        """Loads the model artifact.
        Args:
            artifacts_uri (str):
                Required. The value of the environment variable AIP_STORAGE_URI.
        """
        logging.info("loading model and workflow")
        start_init = time.process_time()
        
        self.model = tf.keras.models.load_model(f"{artifacts_uri}/query_model_merlin")
        self.workflow = nvt.Workflow.load("/workflow/workflow/2t-spotify-workflow")
        self.workflow = self.workflow.remove_inputs(
            [
                'track_pop_can', 
                'track_uri_can', 
                'duration_ms_can', 
                'track_name_can', 
                'artist_name_can',
                'album_name_can',
                'album_uri_can',
                'artist_followers_can', 
                'artist_genres_can',
                'artist_name_can', 
                'artist_pop_can',
                'artist_pop_pl',
                'artist_uri_can', 
                'artists_followers_pl',
            ]
        )
        return self
        
    def predict(self, prediction_input):
        """Preprocesses the prediction input before doing the prediction.
        Args:
            prediction_input (Any):
                Required. The prediction input that needs to be preprocessed.
        Returns:
            The preprocessed prediction input.
        """
        # handle different input types, can take a dict or list of dicts
        self.n_rows = len(prediction_input)
        start = time.process_time()
        pandas_instance = create_pandas_instance(prediction_input)
        #logging.info(f"Pandas conversion took {time.process_time() - start} seconds")
        print(f"Pandas conversion took {time.process_time() - start} seconds")
        start = time.process_time()
        transformed_inputs = nvt.Dataset(pandas_instance)
        #logging.info(f"NVT data loading took {time.process_time() - start} seconds")
        print(f"NVT data loading took {time.process_time() - start} seconds")
        start = time.process_time()
        transformed_instance = self.workflow.transform(transformed_inputs)
        print(f"Workflow transformation took {time.process_time() - start} seconds")

        # def predict(self, instances):
        start = time.process_time()
        
        batch = dataset_to_tensors(transformed_instance)
        print(f"converting to dict_tensors took {time.process_time() - start} seconds")
        start = time.process_time()
        output = self.model(batch)
        print(f"Generating query embeddings took {time.process_time() - start} seconds")
        return transformed_instance, output, batch

Overwriting app/predictor.py


In [310]:
%%writefile app/main.py
from fastapi import FastAPI, Request

import json
import numpy as np
import os
import logging


from google.cloud import storage
from predictor import Predictor

app = FastAPI()

predictor_instance = Predictor()
loaded_predictor = predictor_instance.load(artifacts_uri = os.environ['AIP_STORAGE_URI'])

@app.get(os.environ['AIP_HEALTH_ROUTE'], status_code=200)
def health():
    return {}


@app.post(os.environ['AIP_PREDICT_ROUTE'])
async def predict(request: Request):
    body = await request.json()
    instances = body["instances"]
    outputs = loaded_predictor.predict(instances)

    return {"predictions": outputs[1].numpy().tolist()}

Overwriting app/main.py


In [311]:
%%writefile app/prestart.sh
#!/bin/bash
export PORT=$AIP_HTTP_PORT

Overwriting app/prestart.sh


In [312]:
%%writefile app/instances.json
{"instances": {"collaborative": "false", "album_name_pl": ["There's Really A Wolf", "Late Nights: The Album", "American Teen", "Crazy In Love", "Pony"], "album_uri_can": "spotify:album:5l83t3mbVgCrIe1VU9uJZR", "artist_followers_can": 4339757.0, "artist_genres_can": "'hawaiian hip hop', 'rap'", "artist_genres_pl": ["'hawaiian hip hop', 'rap'", "'chicago rap', 'dance pop', 'pop', 'pop rap', 'r&b', 'southern hip hop', 'trap', 'urban contemporary'", "'pop', 'pop r&b'", "'dance pop', 'pop', 'r&b'", "'chill r&b', 'pop', 'pop r&b', 'r&b', 'urban contemporary'"], "artist_name_can": "Russ", "artist_name_pl": ["Russ", "Jeremih", "Khalid", "Beyonc\u00c3\u00a9", "William Singe"], "artist_pop_can": 82.0, "artist_pop_pl": [82.0, 80.0, 90.0, 87.0, 65.0], "artist_uri_can": "spotify:artist:1z7b1Pr1rSlvWRzsW3HOrS", "artists_followers_pl": [4339757.0, 5611842.0, 15046756.0, 30713126.0, 603837.0], "description_pl": "", "duration_ms_can": 237322.0, "duration_ms_songs_pl": [237506.0, 217200.0, 219080.0, 226400.0, 121739.0], "n_songs_pl": 8.0, "name": "Lit Tunes ", "num_albums_pl": 8.0, "num_artists_pl": 8.0, "track_name_can": "We Just Havent Met Yet", "track_name_pl": ["Losin Control", "Paradise", "Location", "Crazy In Love - Remix", "Pony"], "track_pop_can": 57.0, "track_pop_pl": [79.0, 58.0, 83.0, 71.0, 57.0], "duration_ms_seed_pl": 51023.1, "pid": 1, "track_uri_can": "spotify:track:0VzDv4wiuZsLsNOmfaUy2W", "track_uri_pl": ["spotify:track:4cxMGhkinTocPSVVKWIw0d", "spotify:track:1wNEBPo3nsbGCZRryI832I", "spotify:track:152lZdxL1OR0ZMW6KquMif", "spotify:track:2f4IuijXLxYOeBncS60GUD", "spotify:track:4Lj8paMFwyKTGfILLELVxt"]}}

Overwriting app/instances.json


In [313]:
#make it a package
!touch app/__init__.py

In [314]:
%%writefile Dockerfile

FROM nvcr.io/nvidia/merlin/merlin-tensorflow:23.02
WORKDIR /app 

COPY ./app/requirements.txt /requirements.txt
RUN pip install -r /requirements.txt
#DEBUG CHANGES!!
RUN mkdir /workflow
# RUN mkdir /docker_model
# ADD local_model /docker_model
ADD workflow /workflow
#END DEBUG!


COPY ./app /app
EXPOSE 80
    
CMD ["sh", "-c", "uvicorn main:app --host 0.0.0.0 --port $AIP_HTTP_PORT"]

Overwriting Dockerfile


In [315]:
SERVER_IMAGE = "merlin-prediction-cpr"  # @param {type:"string"} 
REMOTE_IMAGE_NAME=f"{REGION}-docker.pkg.dev/{PROJECT}/{REPOSITORY}/{SERVER_IMAGE}"

!docker build -t $REMOTE_IMAGE_NAME .
# !gcloud builds submit -t $REMOTE_IMAGE_NAME .

Sending build context to Docker daemon  403.1MB
Step 1/9 : FROM nvcr.io/nvidia/merlin/merlin-tensorflow:23.02
 ---> 7f446a9c1777
Step 2/9 : WORKDIR /app
 ---> Using cache
 ---> 59dd9dc6e016
Step 3/9 : COPY ./app/requirements.txt /requirements.txt
 ---> Using cache
 ---> ec60427e1bff
Step 4/9 : RUN pip install -r /requirements.txt
 ---> Using cache
 ---> 8831dddd9dee
Step 5/9 : RUN mkdir /workflow
 ---> Using cache
 ---> e65ff4dd97c9
Step 6/9 : ADD workflow /workflow
 ---> Using cache
 ---> a5aeeaff2858
Step 7/9 : COPY ./app /app
 ---> e205efaab906
Step 8/9 : EXPOSE 80
 ---> Running in b377851ef3ff
Removing intermediate container b377851ef3ff
 ---> 65b2778ea4a6
Step 9/9 : CMD ["sh", "-c", "uvicorn main:app --host 0.0.0.0 --port $AIP_HTTP_PORT"]
 ---> Running in 5dcbaebdcd5e
Removing intermediate container 5dcbaebdcd5e
 ---> df272a52a995
Successfully built df272a52a995
Successfully tagged us-central1-docker.pkg.dev/wortz-project-352116/merlin-spotify-cpr/merlin-prediction-cpr:latest


#### If you are debugging, be sure to set `-d` detached flag off and run the commands in console

us-central1-docker.pkg.dev/wortz-project-352116/merlin-spotify-cpr/merlin-prediction-cp### Copy/paste if you want to run from console for testing
```python
docker run --gpus all -p 80:8080 \
            --name=merlin-prediction-cpr \
            -e AIP_HTTP_PORT=8080 \
            -e AIP_HEALTH_ROUTE=/health \
            -e AIP_PREDICT_ROUTE=/predict \
            -e AIP_STORAGE_URI=gs://spotify-jsw-mpd-2023 \
            us-central1-docker.pkg.dev/wortz-project-352116/merlin-spotify-cpr/merlin-prediction-cpr
```

##### No GPU:
```python
docker run -p 80:8080 \
            --name=merlin-prediction-cpr \
            -e AIP_HTTP_PORT=8080 \
            -e AIP_HEALTH_ROUTE=/health \
            -e AIP_PREDICT_ROUTE=/predict \
            -e AIP_STORAGE_URI=gs://spotify-jsw-mpd-2023 \
            us-central1-docker.pkg.dev/wortz-project-352116/merlin-spotify-cpr/merlin-prediction-cpr
```

#### Test the health route

In [316]:
!docker kill merlin-prediction-cpr
!docker rm merlin-prediction-cpr

merlin-prediction-cpr
E0408 03:51:54.023935357   21953 backup_poller.cc:136]                 Run client channel backup poller: UNKNOWN:pollset_work {created_time:"2023-04-08T03:51:54.023777446+00:00", children:[UNKNOWN:Bad file descriptor {created_time:"2023-04-08T03:51:54.023675071+00:00", errno:9, os_error:"Bad file descriptor", syscall:"epoll_wait"}]}
merlin-prediction-cpr


In [307]:
! curl localhost/health

{}

In [289]:
## Ground truth candidate:
    # 'album_uri_can': 'spotify:album:5l83t3mbVgCrIe1VU9uJZR', 
    # 'artist_name_can': 'Russ', 
    # 'track_name_can': 'We Just Havent Met Yet', 
## TODO - we have to overload with candidate data because of the workflow transform, add overloaded values in the predictor
TEST_INSTANCE = { 'pid': 1,
                 'pl_name_src': 'Lit Tunes ', 
                 'pl_collaborative_src': 'false',
                 'pl_duration_ms_new': 237506.0, 
                  'artist_name_pl': ['Russ', 'Jeremih', 'Khalid', 'Beyonc\xc3\xa9',
                       'William Singe'], 
                 'track_uri_pl': ['spotify:track:4cxMGhkinTocPSVVKWIw0d',
                       'spotify:track:1wNEBPo3nsbGCZRryI832I',
                       'spotify:track:152lZdxL1OR0ZMW6KquMif',
                       'spotify:track:2f4IuijXLxYOeBncS60GUD',
                       'spotify:track:4Lj8paMFwyKTGfILLELVxt'],
                 'track_name_pl': ['Losin Control', 'Paradise', 'Location',
                       'Crazy In Love - Remix', 'Pony'], 
                 'album_name_pl': ["There's Really A Wolf", 'Late Nights: The Album',
                       'American Teen', 'Crazy In Love', 'Pony'], 
                 # 'artist_genres_pl': ["'hawaiian hip hop', 'rap'",
                 #       "'chicago rap', 'dance pop', 'pop', 'pop rap', 'r&b', 'southern hip hop', 'trap', 'urban contemporary'",
                 #       "'pop', 'pop r&b'", "'dance pop', 'pop', 'r&b'",
                 #       "'chill r&b', 'pop', 'pop r&b', 'r&b', 'urban contemporary'"], 
                 # 'artist_pop_can': 82.0, 
                 # 'description_pl': '', 
                 # 'n_songs_pl': 8.0, 
                 
                 # 'num_albums_pl': 8.0, 
                 # 'num_artists_pl': 8.0, 
                 # 'track_pop_pl': [79, 58, 83, 71, 57],
                 # 'duration_ms_seed_pl': 51023.1,
                 
                 
                     }

In [275]:
# Test with an ordered list input (no dictionary)

TEST_INSTANCE_LIST = [TEST_INSTANCE[k] for k in TEST_INSTANCE.keys()]
TEST_INSTANCE_LIST

[1,
 'Lit Tunes ',
 'false',
 237506.0,
 ['Russ', 'Jeremih', 'Khalid', 'BeyoncÃ©', 'William Singe'],
 ['spotify:track:4cxMGhkinTocPSVVKWIw0d',
  'spotify:track:1wNEBPo3nsbGCZRryI832I',
  'spotify:track:152lZdxL1OR0ZMW6KquMif',
  'spotify:track:2f4IuijXLxYOeBncS60GUD',
  'spotify:track:4Lj8paMFwyKTGfILLELVxt'],
 ['Losin Control', 'Paradise', 'Location', 'Crazy In Love - Remix', 'Pony'],
 ["There's Really A Wolf",
  'Late Nights: The Album',
  'American Teen',
  'Crazy In Love',
  'Pony']]

In [323]:
import json
import requests

json_instance = json.dumps({"instances": [TEST_INSTANCE]})
print(json_instance)

{"instances": [{"pid": 1, "pl_name_src": "Lit Tunes ", "pl_collaborative_src": "false", "pl_duration_ms_new": 237506.0, "artist_name_pl": ["Russ", "Jeremih", "Khalid", "Beyonc\u00c3\u00a9", "William Singe"], "track_uri_pl": ["spotify:track:4cxMGhkinTocPSVVKWIw0d", "spotify:track:1wNEBPo3nsbGCZRryI832I", "spotify:track:152lZdxL1OR0ZMW6KquMif", "spotify:track:2f4IuijXLxYOeBncS60GUD", "spotify:track:4Lj8paMFwyKTGfILLELVxt"], "track_name_pl": ["Losin Control", "Paradise", "Location", "Crazy In Love - Remix", "Pony"], "album_name_pl": ["There's Really A Wolf", "Late Nights: The Album", "American Teen", "Crazy In Love", "Pony"]}]}


### Test the predict route

In [324]:
%%time
requests.post('http://localhost/predict', data=json_instance).json()

CPU times: user 2.94 ms, sys: 2.24 ms, total: 5.18 ms
Wall time: 427 ms


{'predictions': [[0.0,
   0.0,
   0.0,
   2.197338342666626,
   0.0,
   0.6343151926994324,
   0.0,
   0.0,
   0.8681452870368958,
   0.082596555352211,
   0.06858403980731964,
   0.8690858483314514,
   0.6754541397094727,
   0.04301293566823006,
   0.0,
   0.0,
   0.0,
   0.11446070671081543,
   3.286738872528076,
   0.0,
   0.0,
   2.3397533893585205,
   0.0,
   1.9221959114074707,
   0.0,
   0.0,
   0.4394824206829071,
   0.0,
   0.8329581618309021,
   0.0,
   0.4763728380203247,
   0.0,
   0.0,
   1.2817498445510864,
   0.0,
   0.0,
   2.144164800643921,
   0.0,
   0.715079665184021,
   0.0,
   0.0,
   0.4205787777900696,
   0.0,
   0.9302131533622742,
   1.045517086982727,
   0.0,
   0.4166524410247803,
   0.0,
   1.2028098106384277,
   0.0,
   0.03395405411720276,
   0.4649958908557892,
   2.633600950241089,
   0.8353680372238159,
   0.0,
   0.6172369122505188,
   0.014025628566741943,
   0.04429948329925537,
   0.0,
   0.0,
   2.771533966064453,
   0.0,
   0.0,
   0.0,
   0.0,
 

## Stop the images if they are running

In [325]:
! docker stop $SERVER_IMAGE
! docker rm $SERVER_IMAGE

merlin-prediction-cpr
merlin-prediction-cpr


### Push the container once ready and testing is complete

In [326]:
# ### push the container to registry
!docker push $REMOTE_IMAGE_NAME

Using default tag: latest
The push refers to repository [us-central1-docker.pkg.dev/wortz-project-352116/merlin-spotify-cpr/merlin-prediction-cpr]

444dc233: Preparing 
018f1149: Preparing 
0f3c1a79: Preparing 
0a88f4b4: Preparing 
625d9539: Preparing 
bd45f47a: Preparing 
959e4900: Preparing 
ed4c0b88: Preparing 
435e7569: Preparing 
6c9c8a80: Preparing 
7ee6b1e6: Preparing 
bd45f47a: Waiting g 
959e4900: Waiting g 
f1d8a22e: Preparing 
7f811af3: Preparing 
fa2a6f77: Preparing 
99f75613: Preparing 
f314ccd3: Preparing 
be9e41cc: Preparing 
042e77d8: Preparing 
b1adf810: Preparing 
5d6e3dc6: Preparing 
5dfe406a: Preparing 
3f5c8bb1: Preparing 
24cd08b1: Preparing 
f287408a: Preparing 
5fc4d7d6: Preparing 
a50a1b1f: Preparing 
13e97a8a: Preparing 
ad3d21ac: Preparing 
09a62824: Preparing 
9a9dae8e: Preparing 
b713171c: Preparing 
1bd92e98: Preparing 
3a6772a2: Preparing 
ce57a041: Preparing 
d4b0d7a5: Preparing 
ed92250c: Preparing 
2ec9aaf4: Preparing 
48aaa07e: Preparing 
84509167: Pr

### Deploy to Vertex AI

After the serving metadata is set below, the model is properly abstracted for use on Vertex

In [327]:
MODEL_DISPLAY_NAME = "Merlin Spotify Query Tower Model - Workflow Local V2"
from google.cloud import aiplatform

ARTIFACT_URI = BUCKET

model = aiplatform.Model.upload(
        display_name=MODEL_DISPLAY_NAME,
        artifact_uri=ARTIFACT_URI,
        serving_container_image_uri=REMOTE_IMAGE_NAME,
        serving_container_predict_route='/predict',
        serving_container_health_route='/health',
        serving_container_command=["sh", "-c", "uvicorn main:app --host 0.0.0.0 --port $AIP_HTTP_PORT"],
        serving_container_args=["--gpus all"],
        sync=True,
    )

Creating Model
Create Model backing LRO: projects/679926387543/locations/us-central1/models/1860842066150424576/operations/701826861944012800
Model created. Resource name: projects/679926387543/locations/us-central1/models/1860842066150424576@1
To use this Model in another session:
model = aiplatform.Model('projects/679926387543/locations/us-central1/models/1860842066150424576@1')


In [ ]:
endpoint = model.deploy(machine_type="n1-standard-8",
                        accelerator_type="NVIDIA_TESLA_T4",
                        accelerator_count=1)

Creating Endpoint
Create Endpoint backing LRO: projects/679926387543/locations/us-central1/endpoints/5440396728375181312/operations/5586543607780737024
Endpoint created. Resource name: projects/679926387543/locations/us-central1/endpoints/5440396728375181312
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/679926387543/locations/us-central1/endpoints/5440396728375181312')
Deploying model to Endpoint : projects/679926387543/locations/us-central1/endpoints/5440396728375181312
Deploy Endpoint model backing LRO: projects/679926387543/locations/us-central1/endpoints/5440396728375181312/operations/9153394512658169856


In [ ]:
#make a prediction

endpoint.predict(instances=[TEST_INSTANCE])

## Finished - now go on to the next notebook to create a [matching engine notebook](03-matching-engine.ipynb) and test out the first end to end recommendation

Be sure to use the output of the endpoint logs above to save the endpoint for use in the matching engine notebook

e.g.:

```python
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/934903580331/locations/us-central1/endpoints/494907775848022016')
```

### Timing analysis

In [125]:
%%timeit
endpoint.predict(instances=[[TEST_INSTANCE, TEST_INSTANCE]])

561 ms ± 95.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [126]:
%%timeit
endpoint.predict(instances=[[TEST_INSTANCE]])

499 ms ± 9.76 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [76]:
endpoint.undeploy_all()

Undeploying Endpoint model: projects/934903580331/locations/us-central1/endpoints/4736674102226452480
Undeploy Endpoint model backing LRO: projects/934903580331/locations/us-central1/endpoints/4736674102226452480/operations/6606761755496415232
Endpoint model undeployed. Resource name: projects/934903580331/locations/us-central1/endpoints/4736674102226452480


resource name: projects/934903580331/locations/us-central1/endpoints/4736674102226452480